In [2]:
from ipynb.fs.full import notes_info

import os
import numpy as np
import librosa
import librosa.display

number of dha :  147
number of ga :  136
number of ma :  130
number of ni :  145
number of pa :  137
number of re :  145
number of sa :  137
number of sa' :  145
-----
mean notes :  140.25
Mean dha sample size: 14234
Mean ga sample size: 14433
Mean ma sample size: 14150
Mean ni sample size: 14000
Mean pa sample size: 15804
Mean re sample size: 12820
Mean sa sample size: 13896
Mean sa' sample size: 15483
-----
Mean sample size: 14346


In [3]:
mean_sample_size = notes_info.mean_sample_size
sample_radius = mean_sample_size//2

In [4]:
main_path = r"C:\Resume_projects\flute_teaching_kit\data\notes"
folder_path = r"C:\Resume_projects\flute_teaching_kit\data\processed_data"

In [ ]:
notes = os.listdir(main_path)

shape = mean_sample_size
shape_correct = []
shape_wrong = []

for note in notes:
    note_path = os.path.join(main_path, note)
    number = 1
    os.makedirs(os.path.join(folder_path, note), exist_ok=True)
    
    for file in os.listdir(note_path):
        file_path = os.path.join(note_path, file)
        y, sr = librosa.load(file_path)

        for start, end in librosa.effects.split(y, top_db=30):
            mid = (start + end) // 2
            left = max(0, mid - sample_radius) 
            right = min(len(y), mid + sample_radius) 

            y_sample = y[left:right]
            y_sample = np.pad(y_sample, (0, max(0, shape - len(y_sample))), mode='constant')[:shape]

            mel_spec = librosa.feature.melspectrogram(y=y_sample, sr=sr, n_mels=128)
            sound_db_sample = librosa.power_to_db(mel_spec, ref=np.max)

            # Clip to range [-80, 0] and normalize to [0, 1]
            sound_db_sample = np.clip(sound_db_sample, -80, 0)
            sound_db_sample = (sound_db_sample + 80) / 80  # Normalize to range [0, 1]

            # Ensure the normalization process (Min-Max scaling)
            min_val = np.min(sound_db_sample)
            max_val = np.max(sound_db_sample)
            if min_val != max_val:  # Avoid division by zero
                sound_db_sample = (sound_db_sample - min_val) / (max_val - min_val)
            else:
                sound_db_sample = np.zeros_like(sound_db_sample)  # Handle case of constant array

            # Save the normalized spectrogram
            np.save(os.path.join(folder_path, note, f"{note}_{number}.npy"), sound_db_sample)
            number += 1